In [1]:
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
import pprint

boc = BrainObservatoryCache(manifest_file='boc/manifest.json')


# Download a list of all targeted areas
targeted_structures = boc.get_all_targeted_structures()
print("all targeted structures: " + str(targeted_structures)+ "\n")
# Download experiment containers for VISp experiments: (targeted_structures=['VISp'])

# Download a list of all imaging depths
depths = boc.get_all_imaging_depths()
print("all imaging depths: " + str(depths) + "\n")

# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print("all cre lines:\n" + str(cre_lines) )

all targeted structures: ['VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISrl']

all imaging depths: [175, 185, 195, 200, 205, 225, 250, 265, 275, 276, 285, 300, 320, 325, 335, 350, 365, 375, 390, 400, 550, 570, 625]

all cre lines:
['Cux2-CreERT2', 'Emx1-IRES-Cre', 'Fezf2-CreER', 'Nr5a1-Cre', 'Ntsr1-Cre_GN220', 'Pvalb-IRES-Cre', 'Rbp4-Cre_KL100', 'Rorb-IRES2-Cre', 'Scnn1a-Tg3-Cre', 'Slc17a7-IRES2-Cre', 'Sst-IRES-Cre', 'Tlx3-Cre_PL56', 'Vip-IRES-Cre']


## VISal

In [2]:
depths = []
for exp in boc.get_experiment_containers(targeted_structures = ['VISal']):
    depths.append(exp['imaging_depth'])
depths = set(depths)
depths

{175, 265, 275, 276, 300, 350, 365, 375}

In [3]:
experiment_containers_df = pd.DataFrame(columns = ['id', 'imaging_depth', 'targeted_structure',
                                 'cre_line','reporter_line','donor_name',
                                 'specimen_name','tags','failed'])

In [4]:
experiment_df = pd.DataFrame(columns = ['id', 'imaging_depth', 'targeted_structure',
                                        'cre_line', 'reporter_line', 'acquisition_age_days',
                                        'experiment_container_id', 'session_type', 'donor_name',
                                        'specimen_name', 'fail_eye_tracking'])

In [5]:
traces_df_dict = {"experiment_id" : [], "traces" : []}

In [ ]:
errors_id = []

In [ ]:
for dep in depths:
    experiment_containers = boc.get_experiment_containers(targeted_structures = ['VISal'],
                                                     imaging_depths = [dep])

    if len(experiment_containers) > 0 :
        trunc = min(2, len(experiment_containers))
        experiment_containers_df2 = pd.DataFrame(experiment_containers[0:trunc], 
                                             columns =['id', 'imaging_depth', 'targeted_structure',
                                                       'cre_line','reporter_line','donor_name',
                                                       'specimen_name','tags','failed'])
        experiment_containers_df = experiment_containers_df.append(experiment_containers_df2)
        
        for exp_cont in range(0,trunc):
            experiment_container_id = experiment_containers[exp_cont]['id']
            exp = boc.get_ophys_experiments(experiment_container_ids=[experiment_container_id])
            
            experiment_df2 = pd.DataFrame(exp,
                                          columns = ['id', 'imaging_depth', 'targeted_structure',
                                        'cre_line', 'reporter_line', 'acquisition_age_days',
                                        'experiment_container_id', 'session_type', 'donor_name',
                                        'specimen_name', 'fail_eye_tracking'])
            experiment_df = experiment_df.append(experiment_df2)
            
            # each experiment has a different session
            for stim in range(0, len(exp)):   
                experiment_id = exp[stim]['id']
                session = exp[stim]['session_type']

                traces_df_dict["experiment_id"].append(experiment_id)
                try:
                    experiment = boc.get_ophys_experiment_data(experiment_id) 
                    experiment_cells_id = experiment.get_cell_specimen_ids()
                    
                    traces_df = pd.DataFrame(experiment.get_dff_traces()[1],
                                            columns = experiment.get_dff_traces()[0],
                                            index = experiment_cells_id )
                    traces_df_dict["traces"].append(traces_df)
                except:
                     errors_id.append(experiment_container_id)               
        print("experiment container n " + str(exp_cont) + "\n")
    print("depth " + str(dep) + "\n")

In [ ]:
errors_id

In [ ]:
experiment_df = experiment_df.loc[~experiment_df['id'].isin(errors_id)]

In [ ]:
experiment_containers_df.to_csv('VISal_experiment_containers_df.txt', 
                                index=None, sep=' ', mode='a')

In [ ]:
experiment_df.to_csv('VISal_experiment_df.txt', 
                      index=None, sep=' ', mode='a')

In [ ]:
for i in range(len(traces_df_dict["traces"])):
    trace_df_name = 'VISal_' + str(traces_df_dict["experiment_id"][i]) + '.txt'
    traces_df_dict["traces"][i].to_csv(trace_df_name, 
                                index=None, sep=' ', mode='a')